In [4]:
import os

In [5]:
%pwd

'e:\\UDEMY PROJECTS\\project1\\research'

In [16]:
os.chdir("../")

In [17]:
%pwd

'e:\\UDEMY PROJECTS\\project1'

In [ ]:
# THIS IS FOR THE ENTITY FOLDER (config_entity.py)
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionconfig:
    root_dir:Path
    source_URL:str
    local_data_file: Path
    unzip_dir:Path

In [21]:
from src.Datascience.constants import *
from src.Datascience.utils.common import read_yaml,create_directories

In [23]:
# This is for the configuration manager
class ConfigurationManager:
    def __init__(self, 
                 config_filepath= CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self)->DataIngestionconfig:
        config = self.config.data_ingestion
        create_directories = ([config.root_dir])

        data_ingestion_config = DataIngestionconfig(
            root_dir= config.root_dir,
            source_URL= config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

        return data_ingestion_config


                      

In [ ]:
# THIS IS FOR THE COMPONENT FOLDER IN WHICH WE WILL CRETE THE DATAINGESTION PYTHON FILE AND WRITE ALL THE BELOW CODE
import os
import urllib.request as request
from src.Datascience import logger
import zipfile

In [38]:
# Data ingestion components
class DataIngestion:
    def __init__(self,config=DataIngestionconfig):
        self.config = config

    # Downloading the zip file 
    # Downloading the zip file
    def download_file(self):
    # Ensure the directory exists before downloading
        os.makedirs(os.path.dirname(self.config.local_data_file), exist_ok=True)
    
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
            url=self.config.source_URL,
            filename=self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with the following info:\n{headers}")
        else:
            logger.info("File already exists")

    
    def extract_zip_file(self):
        """
        zip_file_path :str
        Extract the zip file into the data directory
        function returns None

        """

        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path,exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file,'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [ ]:
# THIS IS FOR THE PIPELINE FOLDER AND IN THAT FOLDER WE WILL MAKE THE DATAINGESTION PIPELINE FILE AND WRITE ALL THE BELOW CODE
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()

except Exception as e:
    raise e

[2024-11-14 14:25:09,351: common: yaml file:config\config.yaml loaded successfully]
[2024-11-14 14:25:09,355: common: yaml file:params.yaml loaded successfully]
[2024-11-14 14:25:09,359: common: yaml file:schema.yaml loaded successfully]
[2024-11-14 14:25:09,362: common: created directory at: artifacts]
[2024-11-14 14:25:10,858: 2008660886: artifacts/data_ingestion/data.zip downloaded with the following info:
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: A1A5:13A1BC:2A052:31476:6735BAF3
Accept-Ranges: bytes
Date: Thu, 14 Nov 2024 08:59:51 GMT
Via: 1.1 varnish
X-Served-By: cache-del21721-DEL
X-Cache: HIT
X-Cache-Hits: 0
X-Timer: S17315747